# Exploratory Data Analysis and Basic Imputation of Missing Values

In [1]:
# import tensorflow as tf
# from tensorflow import keras

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import sklearn

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model as lm
from sklearn import model_selection as ms
sklearn.set_config(print_changed_only=False)

import xgboost

import copy

import import_ipynb
from _Self_Written_Functions_Sheet import rmse

# print('The notebook is running with cuda:', tf.test.is_built_with_cuda())

importing Jupyter notebook from _Self_Written_Functions_Sheet.ipynb


# Data Exploration

In [2]:
# importing the datasets
hp_train = pd.read_csv('..\hp_2a_ranked_edited_train.csv', index_col=0)
hp_test = pd.read_csv('..\hp_2a_ranked_edited_test.csv', index_col=0)
hp_logsaleprice = pd.read_csv('..\hp_1a_no_imputation_logsaleprice.csv', index_col=0)
hp_saleprice = pd.read_csv('..\hp_1a_no_imputation_saleprice.csv')

# saving train index
hp_index = list(hp_train.index)

combo_nonedit = pd.DataFrame(np.concatenate([hp_train, hp_test]))
combo_nonedit.columns = hp_train.columns

for col in combo_nonedit.columns: 
    combo_nonedit[col] = combo_nonedit[col].astype('float', errors='ignore')

combo_cols = list(combo_nonedit.columns)

combo = copy.deepcopy(combo_nonedit)

In [3]:
combo

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,2-STORY 1946 & NEWER,RL,65.0,8450.0,1.0,0.0,Reg,Lvl,3.0,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2003.0,2003.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,4.0,3.0,PConc,4.0,3.0,1.0,6.0,706.0,1.0,0.0,150.0,856.0,GasA,5.0,1.0,5.0,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,4.0,8.0,7.0,0.0,None,Attchd,2003.0,2.0,2.0,548.0,3.0,3.0,3.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,2.0,2008.0,WD,Normal
1,1-STORY 1946 & NEWER ALL STYLES,RL,80.0,9600.0,1.0,0.0,Reg,Lvl,3.0,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6.0,8.0,1976.0,1976.0,Gable,CompShg,MetalSd,MetalSd,None,0.0,3.0,3.0,CBlock,4.0,3.0,4.0,5.0,978.0,1.0,0.0,284.0,1262.0,GasA,5.0,1.0,5.0,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,3.0,6.0,7.0,1.0,TA,Attchd,1976.0,2.0,2.0,460.0,3.0,3.0,3.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,5.0,2007.0,WD,Normal
2,2-STORY 1946 & NEWER,RL,68.0,11250.0,1.0,0.0,IR1,Lvl,3.0,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2001.0,2002.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,4.0,3.0,PConc,4.0,3.0,2.0,6.0,486.0,1.0,0.0,434.0,920.0,GasA,5.0,1.0,5.0,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,4.0,6.0,7.0,1.0,TA,Attchd,2001.0,2.0,2.0,608.0,3.0,3.0,3.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,9.0,2008.0,WD,Normal
3,2-STORY 1945 & OLDER,RL,60.0,9550.0,1.0,0.0,IR1,Lvl,3.0,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7.0,5.0,1915.0,1970.0,Gable,CompShg,Wd Sdng,WdShing,None,0.0,3.0,3.0,BrkTil,3.0,4.0,1.0,5.0,216.0,1.0,0.0,540.0,756.0,GasA,4.0,1.0,5.0,961.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,4.0,7.0,7.0,1.0,Gd,Detchd,1998.0,1.0,3.0,642.0,3.0,3.0,3.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,0.0,None,0.0,2.0,2006.0,WD,Abnorml
4,2-STORY 1946 & NEWER,RL,84.0,14260.0,1.0,0.0,IR1,Lvl,3.0,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8.0,5.0,2000.0,2000.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,4.0,3.0,PConc,4.0,3.0,3.0,6.0,655.0,1.0,0.0,490.0,1145.0,GasA,5.0,1.0,5.0,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,4.0,9.0,7.0,1.0,TA,Attchd,2000.0,2.0,3.0,836.0,3.0,3.0,3.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,12.0,2008.0,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2912,2-STORY PUD - 1946 & NEWER,RM,21.0,1936.0,1.0,0.0,Reg,Lvl,3.0,Inside,Gtl,MeadowV,Norm,Norm,TwnhsI,2Story,4.0,7.0,1970.0,1970.0,Gable,CompShg,CemntBd,CemntBd,None,0.0,3.0,3.0,CBlock,3.0,3.0,1.0,1.0,0.0,1.0,0.0,546.0,546.0,GasA,4.0,1.0,5.0,546.0,546.0,0.0,1092.0,0.0,0.0,1.0,1.0,3.0,1.0,3.0,5.0,7.0,0.0,None,None,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,6.0,2006.0,WD,Normal
2913,2-STORY PUD - 1946 & NEWER,RM,21.0,1894.0,1.0,0.0,Reg,Lvl,3.0,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4.0,5.0,1970.0,1970.0,Gable,CompShg,CemntBd,CemntBd,None,0.0,3.0,3.0,CBlock,3.0,3.0,1.0,3.0,252.0,1.0,0.0,294.0,546.0,GasA,3.0,1.0,5.0,546.0,546.0,0.0,1092.0,0.0,0.0,1.0,1.0,3.0,1.0,3.0,6.0,7.0,0.0,None,CarPort,1970.0,1.0,1.0,286.0,3.0,3.0,3.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0

### Creating functions to use as combinations for trials for linear and tree models

In [4]:
combo.insert(0, 'TotalLivSF', combo['GrLivArea'] + combo['BsmtFinSF1'] + combo['BsmtFinSF2'])

In [5]:
combo[['TotalLivSF']]

,TotalLivSF
0,2416.0
1,2240.0
2,2272.0
3,1933.0
4,2853.0
...,...
2912,1092.0
2913,1344.0
2914,2448.0
2915,1307.0


In [6]:
# functions:

def conditions_edit(df):
    # combine Railroad Adjacent and Railroad Within 200 for both railroads and both condition1 and 2
    df.loc[df['Condition1'].str.contains('RRN', na=False), 'Condition1'] = 'RRN'
    df.loc[df['Condition1'].str.contains('RRA', na=False), 'Condition1'] = 'RRA'

    df.loc[df['Condition2'].str.contains('RRN', na=False), 'Condition2'] = 'RRN'
    df.loc[df['Condition2'].str.contains('RRA', na=False), 'Condition2'] = 'RRA'

def nbrhd_rank(df):
    # creating a dictionary of the neigborhoods and ranks based on median value
    list_neigborhoods = list(df['Neighborhood'].unique())
    n_dict_median = {}
    
    # get the median price for the neighborhoods
    for n in list_neigborhoods:
        n_dict_median[n] = hp_saleprice.loc[hp_train[hp_train['Neighborhood']==n].index].median()[0]
    
    # sorting the dictionary
    n_dict_median_sort = dict(sorted(n_dict_median.items(), key=lambda item:item[1], reverse=True))
    n_list_median_sort = list(n_dict_median_sort.keys())
    n_dict_median_ranking = {}
    
    # assigning ranking
    for i, n in enumerate(n_list_median_sort):
        n_dict_median_ranking[n] = i + 1
        
    # creating a column replacing the neighborhood name with rank
    df['NbMedianRank'] = df['Neighborhood'].replace(n_dict_median_ranking)

def drop_nbrhd(df):
    if 'Neighborhood' in df.columns:
        df.drop('Neighborhood', axis=1, inplace = True)

def totallivsf_add(df):
    # creating a column with total square footage
    df.insert(0, 'TotalLivSF', df['GrLivArea'] + df['BsmtFinSF1'] + df['BsmtFinSF2'])

def drop_grlivarea(df):
    if 'GrLivArea' in df.columns:
        df.drop(['GrLivArea'], axis=1, inplace = True)

def bsmtfin_add(df):
    # adding the column for whether basement is finished
    df.insert(0, 'BsmtFin', df['BsmtFinSF1'] + df['BsmtFinSF2'])
    df.loc[df['BsmtFin'] > 0, 'BsmtFin'] = 1 

def drop_bsmtfinsf12(df):
    if set(['BsmtFinSF1', 'BsmtFinSF2']).issubset(df.columns):
        df.drop(['BsmtFinSF1', 'BsmtFinSF2'], axis=1, inplace = True)

def totalporchsf_add(df):
    # adding the total SF of porches
    df.insert(0, 'TotalPorchSF', df['OpenPorchSF'] + df['EnclosedPorch'] +\
              df['3SsnPorch'] + df['ScreenPorch'])

def drop_porches(df):
    if set(['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']).issubset(df.columns):
        df.drop(['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'], axis=1, inplace = True)

def yrsremodtosold_add(df):
    # calculating the time between remodeling and selling the house
    df.insert(0, 'YrsRemodToSold', df['YrSold'] - df['YearRemodAdd'])

def drop_yrsoldremod(df):
    if set(['YrSold', 'YearRemodAdd']).issubset(df.columns):
        df.drop(['YrSold', 'YearRemodAdd'], axis=1, inplace = True)

def fireplace_yes(df):
    # Replacing number of fireplaces with either yes (1) or no (0)
    df.insert(0, 'Fireplace', [1 if x > 0 else 0 for x in df['Fireplaces']])

def drop_fireplaces(df):
    if 'Fireplaces' in df.columns:
        df.drop(['Fireplaces'], axis=1, inplace = True)

def pool_yes(df): 
    # Replacing pool area with either yes (1) or no (0)
    df.insert(0, 'Pool', [1 if x > 0 else 0 for x in df['PoolArea']])

def drop_poolarea(df):
    if 'PoolArea' in df.columns:
        df.drop(['PoolArea'], axis=1, inplace = True)

def totalbaths_add(df):
    # adding the number of bathrooms
    df.insert(0, 'TotalBaths', df['FullBath'] + df['HalfBath'] * 0.5)

def drop_baths(df):
    if set(['FullBath', 'HalfBath']).issubset(df.columns):
        df.drop(['FullBath', 'HalfBath'], axis=1, inplace = True)
    
def totalbsmtbaths_add(df):
    # adding the number of bathrooms
    df.insert(0, 'TotalBsmtBaths', df['BsmtFullBath'] + df['BsmtHalfBath'] * 0.5)
    
def drop_bsmtbaths(df):
    if set(['BsmtFullBath', 'BsmtHalfBath']).issubset(df.columns):
        df.drop(['BsmtFullBath', 'BsmtHalfBath'], axis=1, inplace = True)

### Creating functions from the models to call them below

In [7]:
def mlr_model(x_trn, y_trn, x_tst, y_tst):
    lm = LinearRegression()
    # parameters to use
    fitintercept = [True, False]
    gparam_mlr_1 = {'fit_intercept': fitintercept}
    # setting parameters
    gs_mlr_1 = ms.GridSearchCV(lm, gparam_mlr_1, cv=n_folds, refit=True, n_jobs=-1,
                               scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_mlr_1.fit(x_trn, y_trn)
    mlr_model.test_rmse = rmse(gs_mlr_1, y_tst, x_tst)    
    
def lasso_model(x_trn, y_trn, x_tst, y_tst):
    lasso = Lasso(random_state=state, max_iter=1000000, selection='random', fit_intercept=True)
    # parameters to use
    alphas = [0.0001, 0.000112, 0.000124, 0.000136, 0.000148]
    gparam_lasso_1 = {'alpha': alphas}
    # setting parameters
    gs_lasso_1 = ms.GridSearchCV(lasso, gparam_lasso_1, cv=n_folds, refit=True, n_jobs=-1,
                                 scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_lasso_1.fit(x_trn, y_trn)
    lasso_model.test_rmse = rmse(gs_lasso_1, y_tst, x_tst)

# def ridge_model(x_trn, y_trn, x_tst, y_tst):
#     ridge = Ridge(random_state=state, max_iter=10000, fit_intercept=True)
#     # parameters to use
#     solvers = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']
#     fitintercept = [True, False]
#     gparam_ridge_1 = {'solver': solvers,
#                       'fit_intercept': fitintercept, 
#                       'random_state': [state]}
#     # setting parameters
#     gs_ridge_1 = ms.GridSearchCV(ridge, gparam_ridge_1, cv=n_folds, refit=True, n_jobs=-1,
#                                  scoring='neg_root_mean_squared_error', return_train_score=True)
#     gs_ridge_1.fit(x_trn, y_trn)
#     ridge_model.test_rmse = rmse(gs_ridge_1, y_tst, x_tst)
    
def enet_model(x_trn, y_trn, x_tst, y_tst):
    enet = ElasticNet(random_state=state, max_iter=10000000,
                      fit_intercept=True, selection='random')
    # parameters to use
    alphas = [1e-4, 2.5e-4, 5e-4]
    l1ratio = [0.5, 0.7, 1]
    precomputes = [True, False]
    warmstart = [True, False]
    gparam_enet_1 = {'alpha': alphas,
                      'l1_ratio': l1ratio,
                      'precompute': precomputes, 
                      'warm_start': warmstart,
                      'random_state': [state]}
    # setting parameters
    gs_enet_1 = ms.GridSearchCV(enet, gparam_enet_1, cv=n_folds, refit=True, n_jobs=-1,
                                scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_enet_1.fit(x_trn, y_trn)
    enet_model.test_rmse = rmse(gs_enet_1, y_tst, x_tst)
    
def rfr_model(x_trn, y_trn, x_tst, y_tst):
    rfr = RandomForestRegressor()
    gparam_rfr = {}
    # setting parameters
    gs_rfr = ms.GridSearchCV(rfr, gparam_rfr, cv=n_folds, refit=True, n_jobs=-1,
                             scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_rfr.fit(x_trn, y_trn)
    rfr_model.test_rmse = rmse(gs_rfr, y_tst, x_tst)
    
def gbm_model(x_trn, y_trn, x_tst, y_tst):
    gbm = GradientBoostingRegressor()
    gparam_gbm = {}
    gs_gbm = GridSearchCV(gbm, gparam_gbm, cv=n_folds, refit=True, n_jobs=-1,
                          scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_gbm.fit(x_trn, y_trn)
    gbm_model.test_rmse = rmse(gs_gbm, y_tst, x_tst)
    
def xgb_model(x_trn, y_trn, x_tst, y_tst):
    xgb_t = xgboost.XGBRegressor()
    gparam_xgb_t = {}
    gs_xgb_t = GridSearchCV(xgb_t, gparam_xgb_t, cv=n_folds, refit=True, n_jobs=-1,
                            scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_xgb_t.fit(x_trn, y_trn)
    xgb_model.test_rmse = rmse(gs_xgb_t, y_tst, x_tst)

In [8]:
def dum_scale():
    # dummify the variables
    hp_dum = pd.get_dummies(combo, drop_first=True)
    enc = OneHotEncoder()
    for feat in ['MoSold', 'YrSold']:
        if feat in hp_dum.columns:
            # encode the feature
            enc.fit(hp_dum[[feat]])
            # create array and then dataframe of the array
            oh_labels = enc.transform(hp_dum[[feat]]).toarray()
            mssubcl_dum = pd.DataFrame(oh_labels)
            mssubcl_dum.columns = enc.get_feature_names_out([feat])
            # Concatenate the dataframes and drop Id and original MSSubClass
            hp_dum = pd.concat([hp_dum, mssubcl_dum], axis=1, sort=False)
            hp_dum = hp_dum.drop([feat], axis=1)
    hp_dum_cols = list(hp_dum.columns)
    for col in hp_dum_cols:
        hp_dum[col] = MinMaxScaler().fit_transform(np.array(hp_dum[col]).reshape(-1,1))
        hp_dum.columns = hp_dum_cols
    dum_scale.hp_dum = copy.deepcopy(hp_dum)

In [ ]:
# getting scores for all models using every possible combination of functions above

hp = copy.deepcopy(combo_nonedit)
state = 0
counter = 0

# using standard kfold split
n_folds = ms.KFold(n_splits=5, random_state=state, shuffle=True)

# lists of the functions separated by whether something is changed or added
stuff_add = [nbrhd_rank, totallivsf_add, bsmtfin_add, totalporchsf_add, 
             yrsremodtosold_add, fireplace_yes, pool_yes, totalbaths_add,
             totalbsmtbaths_add, conditions_edit]

# list of functions that will drop columns
stuff_drop = [drop_nbrhd, drop_grlivarea, drop_bsmtfinsf12,  drop_porches,
              drop_yrsoldremod, drop_fireplaces, drop_poolarea,
              drop_baths, drop_bsmtbaths]

# list of models to use
models = [mlr_model, lasso_model,  enet_model, rfr_model, gbm_model, xgb_model]
# ridge_model,

subset_list = []

# numbers will be indices later down
funcs_scores = {1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}}

import itertools

#https://stackoverflow.com/questions/464864/how-to-get-all-possible-combinations-of-a-list-s-elements
#https://stackoverflow.com/questions/61313027/python-executing-all-permutations-of-list-of-functions

# getting all the combinations of a list for both add and drop
for r_a, r_b in zip(range(len(stuff_add)+1), range(len(stuff_add)+1)):
    com_list_a = list(itertools.combinations(stuff_add, r_a))
    com_list_b = list(itertools.combinations(stuff_drop, r_b))
    
    # getting all permutations of the 2 lists
    for subset in itertools.product(com_list_a, com_list_b):
        
        # adding the names of the functions we run thru to a list to track
        if subset == ((), ()):
            funcs_used_list = ['No changes to dataset']
        else:
            funcs_used_list = str(subset).split()[1::4]

        # if the list of functions is not in the list, run the functions
            # this is mostly to keep track of each permutation
        if funcs_used_list not in subset_list:

            # run the feat eng func
            for subs in subset:
                for s in subs:
                    s(hp)
#                     if callable(s) == True:
#                         s()
#                     else:
#                         continue

            # converting any numbers to numeric type
            for col in combo.columns: 
                combo[col] = combo[col].astype('float', errors='ignore')

            # run dummification and scaling
            dum_scale()

            # getting the train dataset for modeling
            hp = dum_scale.hp_dum.loc[hp_index]

            # getting the functions used to add to dictionary
            funcs_used = ', '.join(str(subset).split()[1::4])
            
            # making the dict key sensible
            if funcs_used == '())':
                funcs_used = 'No changes to dataset'

            # setting up train and test sets
            xtrain, xtest, ytrain, ytest = ms.train_test_split(hp, hp_logsaleprice,
                                                               test_size=0.2, random_state=state)
            # Must flatten to fit
            ytrain = ytrain.values.flatten()

            # run the models
            for i, model in enumerate(models):
                model(xtrain, ytrain, xtest, ytest)
                # add 1 for dictionaries w/in dictionary
                i+=1
                # add scores to dictionary
                funcs_scores[i][funcs_used] = model.test_rmse

            # add the list of functions already run to prevent re-runs
            subset_list.append(funcs_used_list)
            counter += 1
            print(f'Completed round {counter}')
        else:
            continue
            
        # reset the dataset to run the functions on a 'clean' dataset
        hp = copy.deepcopy(combo_nonedit)

Completed round 1
Completed round 2
Completed round 3
Completed round 4
Completed round 5
Completed round 6
Completed round 7
Completed round 8
Completed round 9
Completed round 10
Completed round 11
Completed round 12
Completed round 13
Completed round 14
Completed round 15
Completed round 16
Completed round 17
Completed round 18
Completed round 19
Completed round 20
Completed round 21
Completed round 22
Completed round 23
Completed round 24
Completed round 25
Completed round 26
Completed round 27
Completed round 28
Completed round 29
Completed round 30
Completed round 31
Completed round 32
Completed round 33
Completed round 34
Completed round 35
Completed round 36
Completed round 37
Completed round 38
Completed round 39
Completed round 40
Completed round 41
Completed round 42
Completed round 43
Completed round 44
Completed round 45
Completed round 46
Completed round 47
Completed round 48
Completed round 49
Completed round 50
Completed round 51
Completed round 52
Completed round 53
Co

Completed round 417
Completed round 418
Completed round 419
Completed round 420
Completed round 421
Completed round 422
Completed round 423
Completed round 424
Completed round 425
Completed round 426
Completed round 427
Completed round 428
Completed round 429
Completed round 430
Completed round 431
Completed round 432
Completed round 433
Completed round 434
Completed round 435
Completed round 436
Completed round 437
Completed round 438
Completed round 439
Completed round 440
Completed round 441
Completed round 442
Completed round 443
Completed round 444
Completed round 445
Completed round 446
Completed round 447
Completed round 448
Completed round 449
Completed round 450
Completed round 451
Completed round 452
Completed round 453
Completed round 454
Completed round 455
Completed round 456
Completed round 457
Completed round 458
Completed round 459
Completed round 460
Completed round 461
Completed round 462
Completed round 463
Completed round 464
Completed round 465
Completed round 466


Completed round 827
Completed round 828
Completed round 829
Completed round 830
Completed round 831
Completed round 832
Completed round 833
Completed round 834
Completed round 835
Completed round 836
Completed round 837
Completed round 838
Completed round 839
Completed round 840
Completed round 841
Completed round 842
Completed round 843
Completed round 844
Completed round 845
Completed round 846
Completed round 847
Completed round 848
Completed round 849
Completed round 850
Completed round 851
Completed round 852
Completed round 853
Completed round 854
Completed round 855
Completed round 856
Completed round 857
Completed round 858
Completed round 859
Completed round 860
Completed round 861
Completed round 862
Completed round 863
Completed round 864
Completed round 865
Completed round 866
Completed round 867
Completed round 868
Completed round 869
Completed round 870
Completed round 871
Completed round 872
Completed round 873
Completed round 874
Completed round 875
Completed round 876


Completed round 1226
Completed round 1227
Completed round 1228
Completed round 1229
Completed round 1230
Completed round 1231
Completed round 1232
Completed round 1233
Completed round 1234
Completed round 1235
Completed round 1236
Completed round 1237
Completed round 1238
Completed round 1239
Completed round 1240
Completed round 1241
Completed round 1242
Completed round 1243
Completed round 1244
Completed round 1245
Completed round 1246
Completed round 1247
Completed round 1248
Completed round 1249
Completed round 1250
Completed round 1251
Completed round 1252
Completed round 1253
Completed round 1254
Completed round 1255
Completed round 1256
Completed round 1257
Completed round 1258
Completed round 1259
Completed round 1260
Completed round 1261
Completed round 1262
Completed round 1263
Completed round 1264
Completed round 1265
Completed round 1266
Completed round 1267
Completed round 1268
Completed round 1269
Completed round 1270
Completed round 1271
Completed round 1272
Completed rou

Completed round 1617
Completed round 1618
Completed round 1619
Completed round 1620
Completed round 1621
Completed round 1622
Completed round 1623
Completed round 1624
Completed round 1625
Completed round 1626
Completed round 1627
Completed round 1628
Completed round 1629
Completed round 1630
Completed round 1631
Completed round 1632
Completed round 1633
Completed round 1634
Completed round 1635
Completed round 1636
Completed round 1637
Completed round 1638
Completed round 1639
Completed round 1640
Completed round 1641
Completed round 1642
Completed round 1643
Completed round 1644
Completed round 1645
Completed round 1646
Completed round 1647
Completed round 1648
Completed round 1649
Completed round 1650
Completed round 1651
Completed round 1652
Completed round 1653
Completed round 1654
Completed round 1655
Completed round 1656
Completed round 1657
Completed round 1658
Completed round 1659
Completed round 1660
Completed round 1661
Completed round 1662
Completed round 1663
Completed rou

...

Completed round 40422
Completed round 40423
Completed round 40424
Completed round 40425
Completed round 40426
Completed round 40427
Completed round 40428
Completed round 40429
Completed round 40430
Completed round 40431
Completed round 40432
Completed round 40433
Completed round 40434
Completed round 40435
Completed round 40436
Completed round 40437
Completed round 40438
Completed round 40439
Completed round 40440
Completed round 40441
Completed round 40442
Completed round 40443
Completed round 40444
Completed round 40445
Completed round 40446
Completed round 40447
Completed round 40448
Completed round 40449
Completed round 40450
Completed round 40451
Completed round 40452
Completed round 40453
Completed round 40454
Completed round 40455
Completed round 40456
Completed round 40457
Completed round 40458
Completed round 40459
Completed round 40460
Completed round 40461
Completed round 40462
Completed round 40463
Completed round 40464
Completed round 40465
Completed round 40466
Completed 

Completed round 40795
Completed round 40796
Completed round 40797
Completed round 40798
Completed round 40799
Completed round 40800
Completed round 40801
Completed round 40802
Completed round 40803
Completed round 40804
Completed round 40805
Completed round 40806
Completed round 40807
Completed round 40808
Completed round 40809
Completed round 40810
Completed round 40811
Completed round 40812
Completed round 40813
Completed round 40814
Completed round 40815
Completed round 40816
Completed round 40817
Completed round 40818
Completed round 40819
Completed round 40820
Completed round 40821
Completed round 40822
Completed round 40823
Completed round 40824
Completed round 40825
Completed round 40826
Completed round 40827
Completed round 40828
Completed round 40829
Completed round 40830
Completed round 40831
Completed round 40832
Completed round 40833
Completed round 40834
Completed round 40835
Completed round 40836
Completed round 40837
Completed round 40838
Completed round 40839
Completed 

Completed round 41168
Completed round 41169
Completed round 41170
Completed round 41171
Completed round 41172
Completed round 41173
Completed round 41174
Completed round 41175
Completed round 41176
Completed round 41177
Completed round 41178
Completed round 41179
Completed round 41180
Completed round 41181
Completed round 41182
Completed round 41183
Completed round 41184
Completed round 41185
Completed round 41186
Completed round 41187
Completed round 41188
Completed round 41189
Completed round 41190
Completed round 41191
Completed round 41192
Completed round 41193
Completed round 41194
Completed round 41195
Completed round 41196
Completed round 41197
Completed round 41198
Completed round 41199
Completed round 41200
Completed round 41201
Completed round 41202
Completed round 41203
Completed round 41204
Completed round 41205
Completed round 41206
Completed round 41207
Completed round 41208
Completed round 41209
Completed round 41210
Completed round 41211
Completed round 41212
Completed 

Completed round 41541
Completed round 41542
Completed round 41543
Completed round 41544
Completed round 41545
Completed round 41546
Completed round 41547
Completed round 41548
Completed round 41549
Completed round 41550
Completed round 41551
Completed round 41552
Completed round 41553
Completed round 41554
Completed round 41555
Completed round 41556
Completed round 41557
Completed round 41558
Completed round 41559
Completed round 41560
Completed round 41561
Completed round 41562
Completed round 41563
Completed round 41564
Completed round 41565
Completed round 41566
Completed round 41567
Completed round 41568
Completed round 41569
Completed round 41570
Completed round 41571
Completed round 41572
Completed round 41573
Completed round 41574
Completed round 41575
Completed round 41576
Completed round 41577
Completed round 41578
Completed round 41579
Completed round 41580
Completed round 41581
Completed round 41582
Completed round 41583
Completed round 41584
Completed round 41585
Completed 

Completed round 41914
Completed round 41915
Completed round 41916
Completed round 41917
Completed round 41918
Completed round 41919
Completed round 41920
Completed round 41921
Completed round 41922
Completed round 41923
Completed round 41924
Completed round 41925
Completed round 41926
Completed round 41927
Completed round 41928
Completed round 41929
Completed round 41930
Completed round 41931
Completed round 41932
Completed round 41933
Completed round 41934
Completed round 41935
Completed round 41936
Completed round 41937
Completed round 41938
Completed round 41939
Completed round 41940
Completed round 41941
Completed round 41942
Completed round 41943
Completed round 41944
Completed round 41945
Completed round 41946
Completed round 41947
Completed round 41948
Completed round 41949
Completed round 41950
Completed round 41951
Completed round 41952
Completed round 41953
Completed round 41954
Completed round 41955
Completed round 41956
Completed round 41957
Completed round 41958
Completed 

In [9]:
len(com_list_a)

10

In [9]:
funcs_scores

{1: {'No changes to dataset': 0.11761582220844322},
 2: {'No changes to dataset': 0.10559195823434074},
 3: {'No changes to dataset': 0.1092354290939226},
 4: {'No changes to dataset': 0.12054696319102254},
 5: {'No changes to dataset': 0.11458040396187423},
 6: {'No changes to dataset': 0.1250608887296195}}

In [17]:
for k_1 in funcs_scores:
    k_min = min(funcs_scores[k_1], key=funcs_scores[k_1].get)
    v_min = funcs_scores[k_1][k_min]
    print(k_1)
    print(k_min)
    print(v_min) 
    print('#' * 50)

1
nbrhd_rank, drop_nbrhd
0.11687510094236864
##################################################
2
No changes to dataset
0.10372410349076429
##################################################
3
No changes to dataset
0.10409966566052621
##################################################
4
No changes to dataset
0.12290651009174706
##################################################
5
No changes to dataset
0.11482935489647099
##################################################
6
No changes to dataset
0.1250608887296195
##################################################


In [ ]:
1
No changes to dataset
431179006.78814393
##################################################
2
No changes to dataset
0.10530980944229436
##################################################
3
No changes to dataset
0.10846500826284478
##################################################
4
bsmtfin_add, totalporchsf_add, totalbaths_add, totalbsmtbaths_add, conditions_edit, 
drop_nbrhd, drop_grlivarea, drop_fireplaces, drop_poolarea, drop_baths
0.11746752550813958
##################################################
5
nbrhd_rank, totallivsf_add, totalporchsf_add, yrsremodtosold_add, pool_yes, drop_nbrhd, 
drop_grlivarea, drop_bsmtfinsf12, drop_fireplaces, drop_poolarea
0.11241319030559975
##################################################
6
No changes to dataset
0.12363982116485243
##################################################

In [36]:
funcs_scores

{1: {'No changes to dataset': 0.11761582220844316,
  'nbrhd_rank, drop_nbrhd': 0.11687510094236864,
  'nbrhd_rank, drop_grlivarea': 0.11761582220844413},
 2: {'No changes to dataset': 0.10374124051679229,
  'nbrhd_rank, drop_nbrhd': 0.10721030527532438,
  'nbrhd_rank, drop_grlivarea': 0.10408495485040527},
 3: {'No changes to dataset': 0.10409966566052621,
  'nbrhd_rank, drop_nbrhd': 0.10758634673999228,
  'nbrhd_rank, drop_grlivarea': 0.10439812267347472},
 4: {'No changes to dataset': 0.12435759082044932,
  'nbrhd_rank, drop_nbrhd': 0.12515080070040543,
  'nbrhd_rank, drop_grlivarea': 0.13229286766827406},
 5: {'No changes to dataset': 0.11334023555852867,
  'nbrhd_rank, drop_nbrhd': 0.1148984046183187,
  'nbrhd_rank, drop_grlivarea': 0.1171717519523562},
 6: {'No changes to dataset': 0.1250608887296195,
  'nbrhd_rank, drop_nbrhd': 0.12487345301507552}}

In [37]:
for k_1 in funcs_scores:
    k_min = min(funcs_scores[k_1], key=funcs_scores[k_1].get)
    v_min = funcs_scores[k_1][k_min]
    print(k_1)
    print(k_min)
    print(v_min) 
    print('#' * 50)

1
nbrhd_rank, drop_nbrhd
0.11687510094236864
##################################################
2
No changes to dataset
0.10374124051679229
##################################################
3
No changes to dataset
0.10409966566052621
##################################################
4
No changes to dataset
0.12435759082044932
##################################################
5
No changes to dataset
0.11334023555852867
##################################################
6
nbrhd_rank, drop_nbrhd
0.12487345301507552
##################################################


In [6]:
# saving the imputed AND dummified AND scaled datasets
hp_train_full.to_csv('..\hp_3a_train_tree.csv')
hp_test_full.to_csv('..\hp_3a_test_tree.csv')

# For Linear

In [8]:
# dummify the variables
hp_dum = pd.get_dummies(combo, drop_first=True)

# scaling the dataset
from sklearn.preprocessing import MinMaxScaler
import copy

hp_dum_cols = list(hp_dum.columns)
hp_full = copy.deepcopy(hp_dum)

for col in hp_dum_cols:
    hp_full[col] = MinMaxScaler().fit_transform(np.array(hp_full[col]).reshape(-1,1))
    
hp_train_full = hp_full.loc[hp_index]
hp_test_full = hp_full.loc[1458:]

# saving the imputed AND dummified AND scaled datasets
hp_train_full.to_csv('..\hp_3a_train_linear.csv')
hp_test_full.to_csv('..\hp_3a_test_linear.csv')

In [9]:
# # converting columns containing numbers to numeric type
# for col in combo.columns: 
#     combo[col] = combo[col].astype('float', errors='ignore')
    
# combo.info()